In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime
import numpy as np
from typing import Dict, List, Optional
import json

class DataLogger:
    """Database handler for sensor data and predictions"""
    def __init__(self, db_path: str = "bearing_data.db"):
        self.conn = sqlite3.connect(db_path)
        self.setup_database()

    def setup_database(self):
        """Create necessary tables if they don't exist"""
        with self.conn:
            # Raw sensor data
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS sensor_data (
                    timestamp DATETIME,
                    bearing_id TEXT,
                    vibration REAL,
                    temperature REAL,
                    current REAL,
                    pressure REAL,
                    load REAL
                )
            """)

            # Model predictions
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS predictions (
                    timestamp DATETIME,
                    bearing_id TEXT,
                    pressure_control REAL,
                    lubrication_control REAL,
                    pressure_uncertainty REAL,
                    lubrication_uncertainty REAL,
                    anomaly_score REAL
                )
            """)

            # Maintenance events
            self.conn.execute("""
                CREATE TABLE IF NOT EXISTS maintenance (
                    timestamp DATETIME,
                    bearing_id TEXT,
                    event_type TEXT,
                    description TEXT,
                    technician TEXT
                )
            """)

    def log_sensor_data(self, bearing_id: str, data: Dict[str, float]):
        """Log sensor readings to database"""
        with self.conn:
            self.conn.execute("""
                INSERT INTO sensor_data VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                datetime.now(),
                bearing_id,
                data.get('vibration'),
                data.get('temperature'),
                data.get('current'),
                data.get('pressure'),
                data.get('load')
            ))

    def log_prediction(self, bearing_id: str, prediction: Dict[str, float]):
        """Log model predictions to database"""
        with self.conn:
            self.conn.execute("""
                INSERT INTO predictions VALUES (?, ?, ?, ?, ?, ?, ?)
            """, (
                datetime.now(),
                bearing_id,
                prediction.get('pressure'),
                prediction.get('lubrication'),
                prediction.get('pressure_uncertainty'),
                prediction.get('lubrication_uncertainty'),
                prediction.get('anomaly_score')
            ))

In [ ]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime, timedelta

class BearingDashboard:
    def __init__(self, data_logger: DataLogger):
        self.app = dash.Dash(__name__)
        self.data_logger = data_logger
        self.setup_layout()

    def setup_layout(self):
        """Create dashboard layout"""
        self.app.layout = html.Div([
            html.H1("Bearing Monitoring System"),

            # Bearing selector
            dcc.Dropdown(
                id='bearing-selector',
                options=[
                    {'label': f'Bearing {i}', 'value': f'bearing_{i}'}
                    for i in range(1, 5)
                ],
                value='bearing_1'
            ),

            # Time range selector
            dcc.RadioItems(
                id='timeframe-selector',
                options=[
                    {'label': '1 Hour', 'value': '1H'},
                    {'label': '8 Hours', 'value': '8H'},
                    {'label': '24 Hours', 'value': '24H'},
                    {'label': '7 Days', 'value': '7D'}
                ],
                value='1H'
            ),

            # Real-time monitoring graphs
            html.Div([
                dcc.Graph(id='vibration-graph'),
                dcc.Graph(id='temperature-graph'),
                dcc.Graph(id='control-graph')
            ]),

            # Status indicators
            html.Div([
                html.Div(id='health-status'),
                html.Div(id='maintenance-alert')
            ]),

            # Auto-refresh
            dcc.Interval(
                id='interval-component',
                interval=5*1000,  # 5 seconds
                n_intervals=0
            )
        ])

In [ ]:
import smtplib
from email.mime.text import MIMEText
from typing import List, Dict
import numpy as np

class AlertSystem:
    def __init__(self, email_config: Dict[str, str]):
        self.email_config = email_config
        self.alert_thresholds = {
            'vibration_high': 5.0,
            'temperature_high': 80.0,
            'pressure_low': 100.0,
            'pressure_high': 900.0
        }

    def check_alerts(self, data: Dict[str, float]) -> List[str]:
        """Check sensor data against thresholds"""
        alerts = []

        if data.get('vibration', 0) > self.alert_thresholds['vibration_high']:
            alerts.append(f"High vibration detected: {data['vibration']:.2f}")

        if data.get('temperature', 0) > self.alert_thresholds['temperature_high']:
            alerts.append(f"High temperature detected: {data['temperature']:.2f}°C")

        if data.get('pressure', 0) < self.alert_thresholds['pressure_low']:
            alerts.append(f"Low pressure warning: {data['pressure']:.2f} PSI")

        if data.get('pressure', 0) > self.alert_thresholds['pressure_high']:
            alerts.append(f"High pressure warning: {data['pressure']:.2f} PSI")

        return alerts

    def send_alert(self, message: str, level: str = 'warning'):
        """Send email alert"""
        msg = MIMEText(message)
        msg['Subject'] = f"Bearing Alert - {level.upper()}"
        msg['From'] = self.email_config['from']
        msg['To'] = self.email_config['to']

        with smtplib.SMTP(self.email_config['smtp_server']) as server:
            server.login(
                self.email_config['username'],
                self.email_config['password']
            )
            server.send_message(msg)


In [ ]:
from pymodbus.client.sync import ModbusTcpClient
from typing import Dict

class ModbusInterface:
    """Interface for industrial PLC communication"""
    def __init__(self, host: str, port: int = 502):
        self.client = ModbusTcpClient(host, port)
        self.register_map = {
            'pressure_control': 1000,
            'lubrication_control': 1001,
            'vibration_input': 2000,
            'temperature_input': 2001,
            'current_input': 2002
        }

    def read_sensors(self) -> Dict[str, float]:
        """Read sensor values from PLC"""
        data = {}
        for name, register in self.register_map.items():
            if 'input' in name:
                result = self.client.read_input_registers(register, 1)
                if result.isError():
                    continue
                # Convert from fixed point (value * 100)
                data[name.replace('_input', '')] = result.registers[0] / 100.0
        return data

    def write_controls(self, controls: Dict[str, float]):
        """Write control values to PLC"""
        for name, value in controls.items():
            if name in ['pressure_control', 'lubrication_control']:
                # Convert to fixed point (value * 100)
                register = self.register_map[name]
                self.client.write_register(register, int(value * 100))


In [ ]:
import paho.mqtt.client as mqtt
import json
from typing import Callable, Dict

class MQTTInterface:
    """MQTT interface for IoT deployment"""
    def __init__(self, broker: str, port: int = 1883):
        self.client = mqtt.Client()
        self.client.on_connect = self._on_connect
        self.client.on_message = self._on_message
        self.client.connect(broker, port, 60)
        self.callback = None

    def _on_connect(self, client, userdata, flags, rc):
        """Subscribe to relevant topics on connect"""
        self.client.subscribe("bearing/+/data")
        self.client.subscribe("bearing/+/control")

    def _on_message(self, client, userdata, msg):
        """Handle incoming messages"""
        if self.callback:
            try:
                data = json.loads(msg.payload)
                self.callback(msg.topic, data)
            except json.JSONDecodeError:
                print(f"Invalid JSON received on topic {msg.topic}")

    def set_callback(self, callback: Callable):
        """Set callback for received messages"""
        self.callback = callback

    def publish_control(self, bearing_id: str, control: Dict[str, float]):
        """Publish control commands"""
        topic = f"bearing/{bearing_id}/control"
        self.client.publish(topic, json.dumps(control))

    def start(self):
        """Start MQTT loop"""
        self.client.loop_start()

    def stop(self):
        """Stop MQTT loop"""
        self.client.loop_stop()


In [ ]:
from dataclasses import dataclass
from typing import Dict, Any
import yaml

@dataclass
class SystemConfig:
    """Configuration parameters for the system"""
    # Database settings
    db_path: str = "bearing_data.db"

    # Model settings
    model_path: str = "models/bearing_model.pth"
    device: str = "cuda"

    # Alert settings
    alert_thresholds: Dict[str, float] = None
    email_config: Dict[str, str] = None

    # Communication settings
    modbus_host: str = "localhost"
    modbus_port: int = 502
    mqtt_broker: str = "localhost"
    mqtt_port: int = 1883

    @classmethod
    def from_yaml(cls, path: str) -> 'SystemConfig':
        """Load configuration from YAML file"""
        with open(path, 'r') as f:
            config_dict = yaml.safe_load(f)
        return cls(**config_dict)

# deployment/system_manager.py
from typing import Dict, Any
import threading
import time
import torch
from .modbus_interface import ModbusInterface
from .mqtt_interface import MQTTInterface
from monitoring.alert_system import AlertSystem
from utils.data_collection import DataLogger

class SystemManager:
    """Main system manager coordinating all components"""
    def __init__(self, config: SystemConfig):
        self.config = config

        # Initialize components
        self.model = torch.load(config.model_path)
        self.data_logger = DataLogger(config.db_path)
        self.alert_system = AlertSystem(config.email_config)

        # Initialize communication interfaces
        self.modbus = ModbusInterface(config.modbus_host, config.modbus_port)
        self.mqtt = MQTTInterface(config.mqtt_broker, config.mqtt_port)

        # Set up MQTT callback
        self.mqtt.set_callback(self._handle_mqtt_message)

        # Control loop flag
        self.running = False

    def _handle_mqtt_message(self, topic: str, data: Dict[str, Any]):
        """Handle incoming MQTT messages"""
        if 'data' in topic:
            bearing_id = topic.split('/')[1]
            self.data_logger.log_sensor_data(bearing_id, data)

            # Check for alerts
            alerts = self.alert_system.check_alerts(data)
            for alert in alerts:
                self.alert_system.send_alert(alert)

    def control_loop(self):
        """Main control loop"""
        while self.running:
            try:
                # Read sensor data
                sensor_data = self.modbus.read_sensors()

                # Get model predictions
                with torch.no_grad():
                    predictions = self.model(
                        torch.tensor(sensor_data, device=self.config.device)
                    )

                # Log predictions
                self.data_logger.log_prediction(
                    "bearing_1",  # TODO: Make dynamic
                    predictions
                )

                # Write control outputs
                self.modbus.write_controls({
                    'pressure_control': predictions['pressure'].item(),
                    'lubrication_control': predictions['lubrication'].item()
                })

                time.sleep(0.1)  # 100ms control loop

            except Exception as e:
                print(f"Error in control loop: {e}")
                time.sleep(1)  # Wait before retrying

    def start(self):
        """Start the system"""
        self.running = True
        self.mqtt.start()

        # Start control loop in separate thread
        self.control_thread = threading.Thread(target=self.control_loop)
        self.control_thread.start()

    def stop(self):
        """Stop the system"""
        self.running = False
        self.mqtt.stop()
        self.control_thread.join()


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from typing import Tuple, Dict
from collections import deque

class WaveletLayer(nn.Module):
    """Custom wavelet transform layer for multi-resolution signal analysis"""
    def __init__(self, num_wavelets=32):
        super(WaveletLayer, self).__init__()
        self.num_wavelets = num_wavelets
        # Learnable wavelet parameters
        self.frequencies = nn.Parameter(torch.linspace(1, 100, num_wavelets))
        self.scales = nn.Parameter(torch.ones(num_wavelets))

    def morlet_wavelet(self, t, freq, scale):
        """Morlet wavelet basis function"""
        return torch.exp(1j * freq * t) * torch.exp(-t**2 / (2 * scale**2))

    def forward(self, x):
        batch_size, seq_len = x.shape
        t = torch.linspace(-5, 5, seq_len, device=x.device)

        # Generate wavelet basis
        wavelets = torch.stack([
            self.morlet_wavelet(t, f, s)
            for f, s in zip(self.frequencies, self.scales)
        ])

        # Compute wavelet transforms
        x_complex = x.unsqueeze(1).expand(-1, self.num_wavelets, -1)
        wavelets = wavelets.unsqueeze(0).expand(batch_size, -1, -1)

        # Convolution in frequency domain
        coefficients = torch.fft.fft(x_complex) * torch.fft.fft(wavelets)
        return torch.abs(torch.fft.ifft(coefficients))

class AttentionBlock(nn.Module):
    """Multi-head self-attention with relative positional encoding"""
    def __init__(self, dim, num_heads=8):
        super(AttentionBlock, self).__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.head_dim = dim // num_heads

        self.qkv = nn.Linear(dim, dim * 3)
        self.proj = nn.Linear(dim, dim)

        # Relative positional encoding
        self.rel_pos_enc = nn.Parameter(torch.randn(2 * 128 - 1, self.head_dim))

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        # Compute attention with relative position
        pos_score = self._get_rel_pos(q)
        attn = (q @ k.transpose(-2, -1)) * (1.0 / np.sqrt(self.head_dim))
        attn = attn + pos_score
        attn = F.softmax(attn, dim=-1)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        return x

    def _get_rel_pos(self, q):
        """Compute relative positional encoding"""
        rel_pos_emb = self.rel_pos_enc[127:127+q.size(2)]
        return q @ rel_pos_emb.transpose(0, 1)

class ResidualGatedBlock(nn.Module):
    """Residual block with gating mechanism"""
    def __init__(self, channels):
        super(ResidualGatedBlock, self).__init__()
        self.norm1 = nn.LayerNorm(channels)
        self.conv1 = nn.Conv1d(channels, channels, 3, padding=1)
        self.gate = nn.Sequential(
            nn.Conv1d(channels, channels, 1),
            nn.Sigmoid()
        )
        self.conv2 = nn.Conv1d(channels, channels, 3, padding=1)
        self.norm2 = nn.LayerNorm(channels)

    def forward(self, x):
        residual = x
        x = self.norm1(x.transpose(1, 2)).transpose(1, 2)
        x = self.conv1(x)
        gate = self.gate(x)
        x = x * gate
        x = self.conv2(x)
        x = self.norm2(x.transpose(1, 2)).transpose(1, 2)
        return x + residual

class DynamicThresholdModule(nn.Module):
    """Adaptive thresholding for anomaly detection"""
    def __init__(self, dim, window_size=100):
        super(DynamicThresholdModule, self).__init__()
        self.window_size = window_size
        self.threshold_net = nn.Sequential(
            nn.Linear(dim * 2, dim),
            nn.ReLU(),
            nn.Linear(dim, 1),
            nn.Sigmoid()
        )

    def forward(self, x, history):
        # Compute statistics over history
        mean = torch.mean(history, dim=1, keepdim=True)
        std = torch.std(history, dim=1, keepdim=True)

        # Generate dynamic threshold
        stats = torch.cat([mean, std], dim=-1)
        threshold = self.threshold_net(stats)
        return threshold * 3 * std + mean

class AdvancedBearingControl(nn.Module):
    """Advanced bearing control system with custom architectures"""
    def __init__(self,
                 input_dim=128,
                 hidden_dim=256,
                 num_wavelets=32,
                 num_heads=8,
                 num_layers=4):
        super(AdvancedBearingControl, self).__init__()

        # Signal processing layers
        self.wavelet = WaveletLayer(num_wavelets)
        self.signal_embedding = nn.Sequential(
            nn.Linear(num_wavelets * input_dim, hidden_dim),
            nn.GELU()
        )

        # Sensor fusion with attention
        self.fusion_layers = nn.ModuleList([
            AttentionBlock(hidden_dim, num_heads)
            for _ in range(num_layers)
        ])

        # Processing blocks
        self.process_blocks = nn.ModuleList([
            ResidualGatedBlock(hidden_dim)
            for _ in range(num_layers)
        ])

        # Dynamic thresholding for anomaly detection
        self.threshold_module = DynamicThresholdModule(hidden_dim)

        # Control heads
        self.pressure_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )

        self.lubrication_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 1),
            nn.Sigmoid()
        )

        # Uncertainty estimation
        self.uncertainty_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, 2),  # Uncertainty for pressure and lubrication
            nn.Softplus()
        )

    def forward(self, signal, sensor_data, history=None):
        # Wavelet transform of main signal
        wavelet_coeffs = self.wavelet(signal)
        wavelet_features = wavelet_coeffs.reshape(signal.size(0), -1)

        # Initial embedding
        x = self.signal_embedding(wavelet_features)

        # Attention and processing layers
        for attn, process in zip(self.fusion_layers, self.process_blocks):
            # Attention with sensor fusion
            x = attn(x.unsqueeze(1)) + sensor_data
            x = x.squeeze(1)

            # Residual processing
            x = process(x.unsqueeze(2)).squeeze(2)

        # Generate control outputs with uncertainty
        pressure = self.pressure_head(x)
        lubrication = self.lubrication_head(x)
        uncertainties = self.uncertainty_head(x)

        # Compute dynamic thresholds if history provided
        thresholds = None
        if history is not None:
            thresholds = self.threshold_module(x, history)

        return {
            'pressure': pressure,
            'lubrication': lubrication,
            'uncertainties': uncertainties,
            'thresholds': thresholds,
            'features': x  # For external analysis
        }

class BearingControlSystem:
    def __init__(self, device='cuda' if torch.cuda.is_available() else 'cpu'):
        self.device = device
        self.model = AdvancedBearingControl().to(device)
        self.signal_buffer = deque(maxlen=128)
        self.history_buffer = deque(maxlen=1000)

        # Adaptive learning rate scheduler
        self.optimizer = torch.optim.AdamW(
            self.model.parameters(),
            lr=0.001,
            weight_decay=0.01
        )
        self.scheduler = torch.optim.lr_scheduler.OneCycleLR(
            self.optimizer,
            max_lr=0.01,
            steps_per_epoch=100,
            epochs=10
        )

    def update(self, signal_value: float, sensor_data: Dict[str, float]) -> Dict:
        """Process new data and generate control outputs"""
        self.signal_buffer.append(signal_value)

        if len(self.signal_buffer) < 128:
            return None

        # Prepare inputs
        signal = torch.FloatTensor(list(self.signal_buffer)).unsqueeze(0).to(self.device)
        sensor_tensor = torch.FloatTensor([[
            sensor_data['temperature'],
            sensor_data['current'],
            sensor_data.get('pressure', 0),
            sensor_data.get('load', 0)
        ]]).to(self.device)

        # Get model predictions
        history = torch.FloatTensor(list(self.history_buffer)).unsqueeze(0).to(self.device) if self.history_buffer else None

        with torch.no_grad():
            outputs = self.model(signal, sensor_tensor, history)

        # Update history
        self.history_buffer.append(outputs['features'].cpu().numpy())

        return {
            'pressure': outputs['pressure'].item(),
            'lubrication': outputs['lubrication'].item(),
            'pressure_uncertainty': outputs['uncertainties'][0][0].item(),
            'lubrication_uncertainty': outputs['uncertainties'][0][1].item(),
            'thresholds': outputs['thresholds'].cpu().numpy() if outputs['thresholds'] is not None else None
        }

In [1]:
controller = BearingControlSystem()

# Simulate some data
for _ in range(130):

  signal = np.random.normal(0, 1)
  sensor_data = {
  'temperature': 45 + np.random.normal(0, 2),
  'current': 10 + np.random.normal(0, 0.5),
  'pressure': 500 + np.random.normal(0, 10),
  'load': 1000 + np.random.normal(0, 50)
   }

  outputs = controller.update(signal, sensor_data)
  if outputs:
    print(f"Pressure: {outputs['pressure']:.3f} ± {outputs['pressure_uncertainty']:.3f}")
    print(f"Lubrication: {outputs['lubrication']:.3f} ± {outputs['lubrication_uncertainty']:.3f}")

IndentationError: unexpected indent (<ipython-input-1-52c5c20b918a>, line 14)